In [10]:
import requests
from bs4 import BeautifulSoup
import os
import pdfkit
import PyPDF2

# Define start URL and base URL
start_url = "https://dspy-docs.vercel.app/docs/intro"
base_url = "https://dspy-docs.vercel.app"

# Function to get all links within the website
def get_all_links(start_url, base_url):
    visited = set()
    to_visit = [start_url]
    links = []

    while to_visit:
        url = to_visit.pop(0)
        if url in visited:
            continue
        visited.add(url)
        
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        links.append(url)
        
        for a_tag in soup.find_all('a', href=True):
            link = a_tag['href']
            if link.startswith('/'):
                link = base_url + link
            if link.startswith(base_url) and link not in visited:
                to_visit.append(link)
    
    return links

# Specify the path to wkhtmltopdf executable
path_to_wkhtmltopdf = '/usr/bin/wkhtmltopdf'
config = pdfkit.configuration(wkhtmltopdf=path_to_wkhtmltopdf)

# Function to save HTML as PDF
def save_html_as_pdf(url, output_path):
    try:
        pdfkit.from_url(url, output_path, configuration=config)
    except Exception as e:
        print(f"Failed to convert {url} to PDF: {e}")

# Function to merge multiple PDFs into one
def merge_pdfs(pdf_list, output_path):
    merger = PyPDF2.PdfMerger()

    for pdf in pdf_list:
        merger.append(pdf)

    merger.write(output_path)
    merger.close()

# Get all links to be converted to PDFs
links = get_all_links(start_url, base_url)

# Create a directory to store PDF files
if not os.path.exists('pdfs'):
    os.mkdir('pdfs')

# Convert each link to a PDF
for idx, link in enumerate(links):
    output_path = f'pdfs/page_{idx}.pdf'
    save_html_as_pdf(link, output_path)

# List of PDF files to be merged
pdf_files = [f'pdfs/page_{idx}.pdf' for idx in range(len(links)) if os.path.exists(f'pdfs/page_{idx}.pdf')]

# Merge all PDFs into a single file
merge_pdfs(pdf_files, 'merged_documentation.pdf')


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Failed to convert https://dspy-docs.vercel.app/docs/category/local-language-model-clients to PDF: wkhtmltopdf reported an error:
Exit with code 1 due to network error: ContentNotFoundError

